# Setup Environment

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd
import numpy as np
import torch
import os
import time

device = torch.device("cuda:0")

# Data

In [2]:
retweets = list(np.array(pd.read_csv(os.getcwd()[:os.getcwd().index("SENTICOIN")] + "SENTICOIN/data/raw/twitter/@elonmusk/csv/retweets.csv")["text"]));
print(retweets[:5]);

['@PoisonJL @elonmusk *cough*', '@sethrosson @elonmusk In Papa Elon we trust.', '@RottenAl @dannybuntu @PoisonJL @elonmusk @dogecoin That’s his fault for buying at the peak, by then It was already too late', '@davidgokhshtein @elonmusk SPREAD THE WORD 🚀 🚀🚀 #dogecoin #dogecointo1dollar #dogecointoadollar #DogecoinToTheMoon #dogeArmy https://t.co/04dGPqTS6T', '@elonmusk Elon the people need one more Doge tweet🙏']


# Model: FinBERT for Financial Sentiment Classification

In [3]:
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert").to(device)

# Predicting Sentiment of @elonmusk's Retweets

In [3]:
dtf_retweets = pd.DataFrame(columns = ["text", "Positive", "Negative", "Neutral"]); dtf_retweets

,text,Positive,Negative,Neutral


In [25]:
for i in range(len(retweets)):
    input = tokenizer(retweets[i], padding=True, truncation=True, return_tensors='pt').to(device)
    predictions = torch.nn.functional.softmax(model(**input).logits, dim=-1);

    if torch.cuda.memory_allocated(0)/torch.cuda.memory_reserved(0) * 100 >= 90:
        print("90% GPU Exceeded!");
    if torch.cuda.memory_allocated(0)/torch.cuda.memory_reserved(0) * 100 >= 95:
        print("95% GPU Exceeded!");

    positive = predictions[:, 0].tolist()
    negative = predictions[:, 1].tolist()
    neutral  = predictions[:, 2].tolist()

    table = {"text":retweets[i],
             "Positive":positive,
             "Negative":negative,
             "Neutral" :neutral}

    temp_static_main = pd.DataFrame(table, columns = ["text", "Positive", "Negative", "Neutral"]);
    dtf_retweets = dtf_retweets.append(temp_static_main);

    print(dtf_retweets.drop("text", axis=1).tail(1));
    del input, predictions, temp_static_main

   Positive  Negative  Neutral
0  0.043794  0.093046  0.86316
   Positive  Negative   Neutral
0  0.049379  0.013717  0.936904
   Positive  Negative   Neutral
0  0.023735  0.812563  0.163702
   Positive  Negative   Neutral
0  0.047946  0.038179  0.913875
   Positive  Negative   Neutral
0  0.028746  0.089341  0.881914
   Positive  Negative   Neutral
0  0.041756  0.032347  0.925897
   Positive  Negative   Neutral
0   0.04605  0.070357  0.883593
   Positive  Negative   Neutral
0  0.704448  0.008546  0.287006
   Positive  Negative   Neutral
0  0.021927  0.033836  0.944237
   Positive  Negative   Neutral
0  0.025907  0.095752  0.878342
   Positive  Negative   Neutral
0  0.025774  0.428579  0.545646
   Positive  Negative  Neutral
0  0.069731  0.204779  0.72549
   Positive  Negative   Neutral
0  0.028424  0.513479  0.458097
   Positive  Negative   Neutral
0  0.029951  0.042426  0.927623
   Positive  Negative   Neutral
0  0.021448  0.042614  0.935938
   Positive  Negative   Neutral
0  0.025519 

In [9]:
dtf_retweets.head(5)

,text,Positive,Negative,Neutral
0,@PoisonJL @elonmusk *cough*,0.043794,0.093046,0.863160
1,@sethrosson @elonmusk In Papa Elon we trust.,0.049379,0.013717,0.936904
2,@RottenAl @dannybuntu @PoisonJL @elonmusk @dog...,0.023735,0.812563,0.163702
3,@davidgokhshtein @elonmusk SPREAD THE WORD 🚀 🚀...,0.047946,0.038179,0.913875
4,@elonmusk Elon the people need one more Doge t...,0.028746,0.089341,0.881914


# Save

In [10]:
dtf_retweets.to_csv(os.getcwd()[:os.getcwd().index("SENTICOIN")] + "SENTICOIN/data/processed/models/sentiment/FinBERT.csv", index=False);